# 给定一个净值序列，计算年化收益、最大回撤、夏普比率

##  数据准备：净值序列

~~~ 
以HS300的收盘价，作为策略的净值

~~~

获取数据准备工作

1. 安装JQDdata库，[JQData使用说明](https://www.joinquant.com/help/api/help?name=JQData#%E5%A6%82%E4%BD%95%E5%AE%89%E8%A3%85%E4%BD%BF%E7%94%A8JQData)。
    
2. 在聚宽申请账号，并且[申请开通JQData权限](https://www.joinquant.com/default/index/sdk?utm_campaign=JQData%E7%94%B3%E8%AF%B7&utm_medium=%E7%BD%91%E9%A1%B5&utm_source=%E8%81%9A%E5%AE%BD&gio_link_id=xRxqAjP5)。

3. Python版本: Python3.7

In [1]:
# 从聚宽JQData获取HS300从2015至2018年底，每个交易日的收盘价
# 导入JQData库
from jqdatasdk import *
import pandas as pd

# ID是申请时所填写的手机号；Password为聚宽官网登录密码
auth('user','password') 
pd.set_option('display.max_rows', 10)

# 获取HS300每日收盘价，时间：2015-01-01至2018-12-31
df = get_price('000300.XSHG', start_date='2015-01-01', end_date='2018-12-31', frequency='daily', fields=['close'], skip_paused=False, fq='pre')

# 净值统计：以第一个交易日收盘价为参考，计算每日净值
first_close = df['close'][0]
net_values = df['close']/ first_close
net_values



auth success 


2015-01-05    1.000000
2015-01-06    0.999868
2015-01-07    1.000618
2015-01-08    0.977405
2015-01-09    0.973962
                ...   
2018-12-24    0.834317
2018-12-25    0.828573
2018-12-26    0.824385
2018-12-27    0.821221
2018-12-28    0.826752
Name: close, Length: 975, dtype: float64

In [2]:
import pandas as pd


pd.set_option('display.max_rows', 10)

# 获取HS300每日收盘价，时间：2015-01-01至2018-12-31
df = get_price('000300.XSHG', start_date='2015-01-01', end_date='2018-12-31', frequency='daily', fields=['close'], skip_paused=False, fq='pre')

# 净值统计：以第一个交易日收盘价为参考，计算每日净值
first_close = df['close'][0]
net_values = df['close']/ first_close
net_values

2015-01-05    1.000000
2015-01-06    0.999868
2015-01-07    1.000618
2015-01-08    0.977405
2015-01-09    0.973962
                ...   
2018-12-24    0.834317
2018-12-25    0.828573
2018-12-26    0.824385
2018-12-27    0.821221
2018-12-28    0.826752
Name: close, Length: 975, dtype: float64

In [3]:
df.head()

,close
2015-01-05,3641.54
2015-01-06,3641.06
2015-01-07,3643.79
2015-01-08,3559.26
2015-01-09,3546.72


## 根据日净值分别计算年化收益、最大回撤、夏普比率

###  计算年化收益

- 计算公式
    
    $ Total Annualized Returns=Rp= ((1+P)^{\frac{245}{n}}-1) * 100\%  $ (245是每年的交易日天数)
    
    $ P=策略收益 $

    $ n=策略执行天数 $
    

In [4]:
def compute_annual_profit(days, net_value):
    """
    计算年化收益
    """

    annual_profit = 0
    # 交易日数大于0，才计算年化收益
    if days > 0:
        # 计算年数
        years = days / 245
        # 计算年化收益
        annual_profit = pow(net_value, 1 / years) - 1

    # 将年化收益转化为百分数，保留两位小数
    annual_profit = round(annual_profit * 100, 2)

    return annual_profit

In [5]:
print('年化收益率:', compute_annual_profit(len(df), net_values[-1]), "%")

年化收益率: -4.67 %


### 计算最大回撤

~~~
描述策略可能出现的最糟糕的情况，最极端可能的亏损情况。

~~~


- 计算公式
    
    $ Max Drawdown=Max(\frac{Px−Py}{Px}) $
    
    $ Px,Py=策略某日净值，y>x $ 

In [6]:
def compute_drawdown(net_values):
    """
    计算最大回撤
    :param net_values: 净值列表
    """
    # 最大回撤初始值设为0
    max_drawdown = 0
    index = 0
    # 双层循环找出最大回撤
    for net_value in net_values:
        # 计算从当前开始直到结束，和当前净值相比的最大回撤
        for sub_net_value in net_values[index:]:
            # 计算回撤
            drawdown = 1 - sub_net_value / net_value
            # 如果当前的回撤大于已经计算的最大回撤，则当前回撤作为最大回撤
            if drawdown > max_drawdown:
                max_drawdown = drawdown
        index += 1
    return max_drawdown


In [7]:
max_drawdown = compute_drawdown(list(net_values))
print('最大回撤:', round(max_drawdown * 100, 2), "%")

最大回撤: 46.7 %


In [11]:
def compute_drawdown_oneloop(net_values):
    """
    计算最大回撤
    :param net_values: 净值列表
    """
    # 最大回撤初始值设为0
    max_drawdown = 0
    i = 0
    # 单层循环找出最大回撤
    while i < len(net_values):
        while i+1 < len(net_values) and net_values[i]<net_values[i+1]:
            i += 1
        j = i + 1
        while j < len(net_values) and net_values[j]<net_values[i]:
            if(1-net_values[j]/net_values[i]>max_drawdown):
                max_drawdown = 1-net_values[j]/net_values[i]
            j += 1
        i = j  
    return max_drawdown
max_drawdown = compute_drawdown_oneloop(list(net_values))
print('最大回撤:', round(max_drawdown * 100, 2), "%")

最大回撤: 46.7 %


### 计算夏普比率

~~~
表示每承受一单位总风险，会产生多少的超额报酬，可以同时对策略的收益与风险进行综合考虑。
~~~

- 计算公式
    
    $ Sharpe Ratio = \frac{R_p - R_f}{\sigma_p  } $ 
    
    $ R_p = 策略年化收益率  $
    
    $ R_f = 无风险利率  $
    
    $ \sigma_p = 策略年化收益波动率 = 策略日化收益波动率 \times \sqrt{245}   $
    
  
    
    


In [7]:
import pandas as pd

def compute_sharpe_ratio(net_values):
    """
    计算夏普比率
    :param net_values: 净值列表
    """

    # 总交易日数
    trading_days = len(net_values)
    # 所有收益的DataFrame
    profit_df = pd.DataFrame(columns={'profit'})
    # 收益之后，初始化为第一天的收益
    profit_df.loc[0] = {'profit': round((net_values[0] - 1) * 100, 2)}
    # 计算每天的收益
    for index in range(1, trading_days):
        # 计算每日的收益变化
        profit = (net_values[index] - net_values[index - 1]) / net_values[index - 1]
        profit = round(profit * 100, 2)
        profit_df.loc[index] = {'profit': profit}

    

    # 计算当日收益标准差
    profit_std = pow(profit_df['profit'].var(), 1 / 2)

    # 年化收益
    annual_profit = compute_annual_profit(trading_days, net_values[-1])

    # 夏普比率
    sharpe_ratio = (annual_profit - 4.75) / (profit_std * pow(245, 1 / 2))

    return sharpe_ratio



In [8]:
sharp = compute_sharpe_ratio(list(net_values))
print('夏普比率:', round(sharp,3))

夏普比率: -0.375


# 给定一个投资组合的收益序列，以沪深300作为参照，分解该投资组合的alpha和beta

## 获取数据, 计算收益序列

~~~ 
以ZZ500ETF的收盘价，作为投资组合的净值序列

~~~

In [9]:
import numpy as np
import pandas as pd
import tushare as ts


target="510500.XSHG" # 中证500ETF
benchmark="000300.XSHG" # 沪深300指数
start="2015-1-1"
end="2018-12-31"

p_target = get_price(target, start_date=start, end_date=end, frequency='daily', fields=['close'], skip_paused=False, fq='pre')
p_benchmark = get_price(benchmark, start_date=start, end_date=end, frequency='daily', fields=['close'], skip_paused=False, fq='pre')


# 为避免投资组合的净值(价格)采样时间与基准的采样时间不同, 取两者的交集
price= pd.concat([p_target, p_benchmark], axis=1, join='inner') 
price.columns=[target,benchmark]

# 投资组合净值的时间顺序, 与参照基准的时间顺序, 这两者可能不同, 因此统一到时间降序上
price.sort_index(ascending=False, inplace=True)
price.head()

,510500.XSHG,000300.XSHG
2018-12-28,4.451,3010.65
2018-12-27,4.445,2990.51
2018-12-26,4.501,3002.03
2018-12-25,4.517,3017.28
2018-12-24,4.577,3038.20


In [10]:
# 计算收益
r= (price-price.shift(-1))/price.shift(-1)
r.drop(r.index[-1], inplace=True) 
r.head()

,510500.XSHG,000300.XSHG
2018-12-28,0.001350,0.006735
2018-12-27,-0.012442,-0.003837
2018-12-26,-0.003542,-0.005054
2018-12-25,-0.013109,-0.006886
2018-12-24,0.008594,0.002905


## 计算$ \beta $

~~~
表示投资的系统性风险。
~~~

- 计算公式
    
    $ \beta = \frac{Cov(r_P, r_B)} {Var(r_B)} $
        
    $ r_P = 投资组合P的超额收益率 = 投资组合每日收益率 - 日化无风险利率 $
    
    $ r_B = 业绩基准B的超额收益率 = 基准每日收益率 - 日化无风险利率  $
    

In [16]:
riskfree = 0.02/245  # 日化无风险收益
r_p = r[target]- riskfree
r_b = r[benchmark]- riskfree
beta=np.cov(r_p, r_b)[0,1]/np.var(r_b)
beta


1.1407630051106075

## 计算 $\alpha $ 

~~~
  投资中面临着系统性风险（即Beta）和非系统性风险（即Alpha），Alpha是投资者获得与市场波动无关的回报。
~~~

- 计算公式

    $ Alpha = \alpha = R_p - [R_f + \beta_p \times (R_b - R_f)])$
    
    $ R_p = 投资组合P的年化收益率 $
    
    $ R_b = 业绩基准B的年化收益率 $
    
    $ R_f = 无风险利率(年化) $
    
    $ \beta_p = 投资组合P的\beta值 $

In [31]:
Rf = 0.02   # 无风险利率(年化)
profit =  (price[target][-1] - price[target][0])/price[target][0]

# compute_annual_profit返回年化收益率的单位是%, 这里需要先转换成普通数值
Rp = compute_annual_profit(len(price[target]), profit)/100

profit =  (price[benchmark][-1] - price[benchmark][0])/price[benchmark][0]
Rb = compute_annual_profit(len(r[benchmark]), profit)/100
alpha= Rp - (Rf + beta*(Rb - Rf))
alpha

0.051363236763159625

## 习题一：根据HS300周净值序列， 分别计算年化收益、最大回撤、夏普比率

In [13]:
# 样例： 从聚宽JQData获取HS300的周K数据

from jqdatasdk import *
import pandas as pd

# ID是申请时所填写的手机号；Password为聚宽官网登录密码
auth('ID','Password') 

pd.set_option('display.max_rows', 10)

# 获取HS300周K收盘价
df = get_price('000300.XSHG', start_date='2015-01-01', end_date='2018-12-31', frequency='5d', fields=['close'], skip_paused=False, fq='pre')
first_close = df['close'][0]
net_values = df['close']/ first_close
net_values

ModuleNotFoundError: No module named 'jqdatasdk'

## 习题二：使用ZZ500的周净值序列做为投资组合的收益序列，以HS300作为参照，分解alpha和beta。

(建议，也可选用其他指数的日（或周）净值序列作为投资组合的收益序列，例如：上证50ETF，HS300ETF等)